In [26]:
%matplotlib inline
from scipy import stats
from pathlib import Path
import pandas as pd
import numpy as np
from scipy.stats import trim_mean
from statsmodels import robust
import wquantiles
import seaborn as sns
import matplotlib.pylab as plt

# Organizing Data

In [69]:
metro_data = pd.read_csv("afluenciastc_simple_12_2022.csv", encoding='latin')
#print(influx_metro.head(15))
metro_data_line_1 = metro_data[metro_data.linea == "Linea 1"]
metro_data_line_1_Zara = metro_data_line_1[metro_data_line_1.estacion == "Zaragoza"]
print(metro_data_line_1_Zara.head(30))
metro_influx = metro_data_line_1_Zara

           fecha  anio    mes    linea  estacion  afluencia
0     2010-01-01  2010  Enero  Linea 1  Zaragoza      20227
195   2010-01-02  2010  Enero  Linea 1  Zaragoza      50405
465   2010-01-03  2010  Enero  Linea 1  Zaragoza      24618
657   2010-01-04  2010  Enero  Linea 1  Zaragoza      60654
780   2010-01-05  2010  Enero  Linea 1  Zaragoza      66837
1047  2010-01-06  2010  Enero  Linea 1  Zaragoza      51980
1170  2010-01-07  2010  Enero  Linea 1  Zaragoza      57813
1490  2010-01-08  2010  Enero  Linea 1  Zaragoza      55981
1641  2010-01-09  2010  Enero  Linea 1  Zaragoza      48085
1873  2010-01-10  2010  Enero  Linea 1  Zaragoza      30171
2065  2010-01-11  2010  Enero  Linea 1  Zaragoza      65649
2263  2010-01-12  2010  Enero  Linea 1  Zaragoza      44332
2340  2010-01-13  2010  Enero  Linea 1  Zaragoza      49462
2604  2010-01-14  2010  Enero  Linea 1  Zaragoza      53209
2730  2010-01-15  2010  Enero  Linea 1  Zaragoza      60354
3115  2010-01-16  2010  Enero  Linea 1  

# Measures of centrality (Mean, Median)


In [57]:
metro_influx = metro_data["afluencia"]
#Getting the mean of the dataframe, however it is not entirely correct
mean = metro_influx.mean()
#Median
median = metro_influx.median()
print(f"Mean = {mean}")
print("Median =",median)

         fecha  anio    mes    linea             estacion  afluencia
0   2010-01-01  2010  Enero  Linea 1             Zaragoza      20227
1   2010-01-01  2010  Enero  Linea 1  Isabel la CatÃ³lica       6487
2   2010-01-01  2010  Enero  Linea 1            Moctezuma      10304
3   2010-01-01  2010  Enero  Linea 1         Pino SuÃ¡rez       8679
4   2010-01-01  2010  Enero  Linea 1       GÃ³mez FarÃ­as      19499
5   2010-01-01  2010  Enero  Linea 6  Deptvo. 18 de Marzo        621
6   2010-01-01  2010  Enero  Linea 6    La Villa-Basilica      24792
7   2010-01-01  2010  Enero  Linea 9           PantitlÃ¡n      27000
8   2010-01-01  2010  Enero  Linea 8               Aculco       3652
9   2010-01-01  2010  Enero  Linea 9           VelÃ³dromo       3239
10  2010-01-01  2010  Enero  Linea 5  Autobuses del Norte      16824
11  2010-01-01  2010  Enero  Linea 5            Misterios       3513
12  2010-01-01  2010  Enero  Linea 7       Constituyentes       1417
13  2010-01-01  2010  Enero  Linea

# Measures of variability (STD, IQR, MAD)

In [49]:
#Standard Deviation
standard_deviation = metro_influx.std()
#Interquartile range
IQR = metro_influx.quantile(0.75) - metro_influx.quantile(0.25)
#Median Absolute Deviation (MAD)
MAD = robust.scale.mad(metro_influx)
print(f"Standard Deviation = {standard_deviation}")
print(f"IQR = {IQR}")
print("MAD =",MAD)


Standard Deviation = 20198.750548652955
IQR = 18839.0
MAD = 12751.861681366683


# Exploring data

In [70]:
#metro_influx.quantile([0.05,0.25,0.5,0.75,0.95])
#Boxplot
ax = (metro_influx/1000).plot.box()
ax.set_ylabel("Influx (thousands)")
binnedInflux = pd.cut(metro_inlux, 10)
binnedInflux.value_counts()

TypeError: unsupported operand type(s) for /: 'str' and 'int'